# Example: Using Pandas to Analyze Completion Parameters

This notebook illustrates using the Python API and the pandas package to perform completion analysis.

## 0.5 Import packages

The only import needed for the Python API is `orchid` itself.

In [ ]:
import orchid

The remaining imports are standard python packages to support the analysis.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate

## 1.0 Load the .ifrac project

In [ ]:
project = orchid.load_project(
    r'c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.ifrac')

## 2.0 Define a function to compute the stage treatment data

In [ ]:
def compute_stage_treatment_aggregates(treatment_stage):
    
    def slurry_rate_per_min_to_per_second_conversion_factor():
        source_slurry_rate_unit = treatment_curves['Slurry Rate'].sampled_quantity_unit()
        target_slurry_rate_unit = \
            f'{orchid.slurry_rate_volume_unit(source_slurry_rate_unit)}/s'
        local_result = orchid.get_conversion_factor(source_slurry_rate_unit,
                                                    target_slurry_rate_unit)
        return local_result

    def slurry_rate_bbl_per_second_to_gal_per_second_conversion_factor():
        local_result = orchid.get_conversion_factor('bbl/s', 'gal/s')
        return local_result

    treatment_curves = treatment_stage.treatment_curves()
    if not treatment_curves:
        return None
    
    raw_treatment_series = {n: c.time_series() for n, c in (treatment_curves.items())}
    stage_start_time = np.datetime64(stage.start_time)
    stage_stop_time = np.datetime64(stage.stop_time)
    
    fluid_per_sec = \
        (raw_treatment_series[orchid.SLURRY_RATE][stage_start_time:stage_stop_time] *
         slurry_rate_per_min_to_per_second_conversion_factor())
    stage_fluid = integrate.trapz(fluid_per_sec.values, 
                                  (fluid_per_sec.index - stage_start_time).seconds)
    
    stage_concentration = \
        raw_treatment_series[orchid.PROPPANT_CONCENTRATION][stage_start_time:stage_stop_time]
    proppant_per_sec = (fluid_per_sec * 
                        slurry_rate_bbl_per_second_to_gal_per_second_conversion_factor() *
                        stage_concentration)
    stage_proppant = integrate.trapz(proppant_per_sec.values, 
                                     (proppant_per_sec.index - stage_start_time).seconds)
    
    stage_pressure = \
        raw_treatment_series[orchid.TREATING_PRESSURE][stage_start_time:stage_stop_time]
    median_stage_pressure = stage_pressure.median()
    
    # Assumes that all three curves have the same time basis; that is, 
    # that the index for each series is equal.
    treatment_curves = pd.DataFrame.from_dict(raw_treatment_series)
    treatment_curves['dt'] = ((treatment_curves.index.values - stage_start_time) /
                              np.timedelta64(1, 's'))
    
    return stage_fluid, stage_proppant, median_stage_pressure

## 3.0 Build a pandas data frame

In [ ]:
# Remember the project units
units = {'length': project.unit('length'),
         'mass': project.unit('mass'),
         'pressure': project.unit('pressure'),
         'slurry volume': orchid.slurry_rate_volume_unit(project.unit('slurry rate')),
         'proppant mass': orchid.proppant_concentration_mass_unit(
             project.unit('proppant concentration'))}

In [ ]:
# Calculate the stage results
stage_results = []
for well in project.wells:
    stages = list(well.stages)

    for stage in stages:
        
        treatment_aggregates = compute_stage_treatment_aggregates(stage)
        # Skip stages with no aggregates. These stages most likely are from 
        # an untreated monitor well.
        if not treatment_aggregates:
            continue
            
        stage_fluid, stage_proppant, median_stage_pressure = treatment_aggregates
        stage_results.append((project.name, well.name, stage.display_stage_number,
                              stage.md_top(units['length']).magnitude,
                              stage.md_bottom(units['length']).magnitude,
                              stage_fluid, stage_proppant, median_stage_pressure))

In [ ]:
# Provide a way to manage DataFrame column names (which include units) using simpler,
# semantic identifiers. Creating the column names helps me avoid "typos" involved in
# getting the (project-specific) units correct, but is not necessary in a typical
# interactive session.

name_column_name_map = {'project': 'Project',
                        'well': 'Well',
                        'stage': 'Stage',
                        'md_top': f'MD Top ({units["length"]})',
                        'md_bottom': f'MD Bottom ({units["length"]})',
                        'total_fluid': f'Total Fluid ({units["slurry volume"]})',
                        'total_proppant': f'Total Proppant ({units["proppant mass"]})',
                        'median_treating': f'Median Treating Pressure ({units["pressure"]})'}

In [ ]:
# Create the data frame
stage_summaries = pd.DataFrame(data=stage_results,
                               columns=name_column_name_map.values())
stage_summaries.head()

### 3.1 Compute the stage length directly from the data frame

In [ ]:
stage_summaries[f'Stage Length ({units["length"]})'] = \
    stage_summaries.apply(
        lambda s: s[name_column_name_map['md_bottom']] -
                  s[name_column_name_map['md_top']], axis=1)
name_column_name_map['stage_length']= stage_summaries.columns[-1]
stage_summaries.head()

### 3.2 Now compute the proppant loading for each stage

In [ ]:
stage_summaries[f'Proppant loading ({units["proppant mass"]}/{units["length"]})'] = \
    stage_summaries.apply(
        lambda s: s[name_column_name_map['total_proppant']] /
                  s[name_column_name_map['stage_length']], axis=1)
name_column_name_map['proppant_loading']= stage_summaries.columns[-1]
stage_summaries.head()

## 4.0 Completion questions

### 4.1 What is the median proppant intensity per well?

In [ ]:
stage_summaries[[name_column_name_map['well'],
                 name_column_name_map['proppant_loading']]].\
    groupby(name_column_name_map['well']).median()

In [ ]:
groups = stage_summaries.groupby(name_column_name_map['well'])

fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group[name_column_name_map['stage']],
            group[name_column_name_map['proppant_loading']],
            marker='o', linestyle='', ms=6, label=name)
ax.legend()
plt.rcParams['figure.dpi'] = 150
plt.show()

That's a little hard to interpret with the outliers

In [ ]:
# Plot
groups = stage_summaries[stage_summaries[name_column_name_map['stage']]>5].\
    groupby(name_column_name_map['well'])

fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group[name_column_name_map['stage']],
            group[name_column_name_map['proppant_loading']],
            marker='o', linestyle='', ms=6, label=name)
ax.legend()
ax.set_title(f'Proppant Loading by Stage')
ax.set_xlabel('Stage Number')
ax.set_ylabel(name_column_name_map['proppant_loading'])
plt.rcParams['figure.dpi'] = 150
plt.show()

That's a little busy. Let's clean it up.

In [ ]:
def build_proppant_loading_plot(data_frame):
    # Plot
    groups = data_frame.groupby(name_column_name_map['well'])

    fig, ax = plt.subplots(len(groups), sharex=True, sharey=True)
    fig.suptitle(f'{name_column_name_map["proppant_loading"]} by Stage')
    i=0
    colors=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple',
            'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
    for name, group in groups:
        ax[i].margins(0.05)
        ax[i].plot(group[name_column_name_map['stage']],
                   group[name_column_name_map['proppant_loading']],
                   marker='o', linestyle='', ms=6, label=name)
        ax[i].legend()
        ax[i].set_xlabel('Stage Number')
        i = i+1
        # Hide x labels and tick labels for all but bottom plot.
    for a in ax:
        a.label_outer()
    plt.rcParams['figure.dpi'] = 150
    plt.show()

build_proppant_loading_plot(stage_summaries[stage_summaries[name_column_name_map['stage']]>5])